In [1]:
import tensorflow as tf
from tensorflow import keras

C:\Users\HP\new_anacoda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\new_anacoda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\new_anacoda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\new_anacoda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
import numpy as np

In [3]:
data=keras.datasets.imdb

In [4]:
word=data.get_word_index()
word

1646592/1641221 [==============================] - 17s 10us/step


{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [5]:
(train_data,train_labels),(test_data,test_labels)=data.load_data(num_words=10000)

17465344/17464789 [==============================] - 330s 19us/step


# preproceesing and preparing the data ready for the modelling

In [6]:
word={k:v+3 for k,v in word.items()}
word["<PAD>"]=0
word["<START>"]=1
word["<UNK>"]=2
word["<UNUSED>"]=3
reversed_word=dict([(value,key) for (key,value) in word.items()]) 

# visualising the review

In [7]:
def decode(text):
    return " ".join([reversed_word.get(i,'?') for i in text])
print(decode(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

# making every review length to 250

In [9]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word["<PAD>"],padding="post",maxlen=250)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word["<PAD>"],padding="post",maxlen=250)


In [10]:
len(test_data[0]),len(train_data[0])

(250, 250)

# building a model

In [11]:
model=keras.Sequential()
model.add(keras.layers.Embedding(10000,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


# compiling and training

In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# preparing validation data

In [14]:
x_val=train_data[:10000]
xtrain=train_data[10000:]
y_val=train_labels[:10000]
ytrain=train_labels[10000:]


In [16]:
model.fit(xtrain,ytrain,epochs=50,batch_size=512,validation_data=(x_val,y_val),verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/50
15000/15000 [==============================] - 2s 126us/sample - loss: 0.6915 - acc: 0.5151 - val_loss: 0.6888 - val_acc: 0.5867
Epoch 2/50
15000/15000 [==============================] - 1s 95us/sample - loss: 0.6839 - acc: 0.6889 - val_loss: 0.6792 - val_acc: 0.7007
Epoch 3/50
15000/15000 [==============================] - 1s 99us/sample - loss: 0.6696 - acc: 0.7377 - val_loss: 0.6620 - val_acc: 0.7507
Epoch 4/50
15000/15000 [==============================] - 1s 97us/sample - loss: 0.6466 - acc: 0.7571 - val_loss: 0.6365 - val_acc: 0.7701
Epoch 5/50
15000/15000 [==============================] - 1s 97us/sample - loss: 0.6144 - acc: 0.7905 - val_loss: 0.6036 - val_acc: 0.7895
Epoch 6/50
15000/15000 [==============================] - 2s 109us/sample - loss: 0.5749 - acc: 0.8155 - val_loss: 0.5658 - val_acc: 0.8034
Epoch 7/50
15000/15000 [==============================] - 2s 116us/sample - loss: 0.5312 - acc: 0.8306 - val_loss:

In [17]:
predicted=model.predict(test_data)

In [23]:
cal=[]
for i in predicted:
    if(i>0.5):
        cal.append(1)
    else:
        cal.append(0)

In [26]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_labels,cal)
cm

array([[10883,  1617],
       [ 1727, 10773]], dtype=int64)

In [31]:
testaccuracy=(cm[0][0]+cm[1][1])/(sum(cm[0])+sum(cm[1]))
testaccuracy=testaccuracy*100
testaccuracy

86.624

# checking the review

In [33]:
rev=test_data[0]
print(decode(rev))
print('predicted val= ',cal[0])
print('actual val = ',test_labels[0])


<START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite <UNK> so all you madison fans give this a miss <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

# giving our own review to predict

In [41]:
def encode_review(s):
    encode=[1]
    s=s.split()
    for i in s:
        if i in word:
            encode.append(word[i])
        else:
            encode.append(2)
    return encode
            
    

In [75]:
revi='the movie was so good with nice adventures I want to go and watch It was really once in a life time experience'
encoded=[encode_review(revi)]

In [76]:
encoded=keras.preprocessing.sequence.pad_sequences(encoded,value=word["<PAD>"],padding="post",maxlen=250)

In [78]:
model.predict(encoded)

array([[0.6455814]], dtype=float32)

this indocates our review is 64.5% postive